In [82]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
import sys
import time

In [27]:
def get_histogram(N):
  # Calculamos el histograma de los píxeles del bloque
  histogram = cv2.calcHist([N], [0], None, [256], [0, 256])

  # Devolvemos el histograma como un vector
  return histogram.flatten()

In [3]:
def im_to_col(imagen, m, n):
    filas, columnas = imagen.shape
    a = m//2
    b = n//2
    imagen2 = np.zeros((m*n, filas*columnas))
    imagen_amp = cv2.copyMakeBorder(imagen, a, a, b, b, cv2.BORDER_REPLICATE)
    aux = 0
    for i in range(a, filas+a):
        for j in range(b, columnas+b):
            imagen2[:, aux] = imagen_amp[i-a:i+a+1, j-b:j+b+1].flatten()
            aux += 1
    return imagen2

In [54]:
def feature_extraction2(image, N , R):
  # Obtenemos las dimensiones de la imagen
  height, width = image.shape[:2]  

  print("IMAGEN NxM: ",height, "x",width )
 
 

  # Calculamos el número de bloques en filas y columnas
  num_blocks_rows = height // N
  num_blocks_cols = width // N

  # Dividimos la imagen en bloques de 16 filas
  blocks = np.vsplit(image, num_blocks_rows)

  # Creamos una lista para guardar todas las celdas
  celdas = []

  # Iteramos sobre los bloques
  for i, block in enumerate(blocks):
    # Dividimos cada bloque en bloques de 16 columnas
    cells = np.hsplit(block, num_blocks_cols)

    # Iteramos sobre las celdas
    for j, cell in enumerate(cells):
      # Añadimos cada celda a la lista
      celdas.append(cell)
  
  celdas = np.array(celdas)
  print("TENEMOS: ",len(celdas), " celdas")
  histograms = np.zeros((len(celdas) , N*N))

  for i, celda in enumerate(celdas):    
    vecinos = im_to_col(celda,R,R).T

    result = np.zeros(N*N, np.uint8)

    for j, v in enumerate(vecinos):
      v = v.reshape(R,R)  

      center_pixel = v[1,1]   
      # Inicializamos el valor binario a 0
      binary = ''   

      for x in range(R):
        for y in range(R):  

          # Saltamos el píxel central          
          if x == 1 and y == 1:
            continue             

          # Actualizamos el valor binario en función del vecino
          if v[x, y] < center_pixel:
            binary += '0'
          else:
            binary += '1'

      # Modificamos el valor del píxel central por el del número binario obtenido
      result[j] = int(binary,2)

    # Calculamos el histograma del bloque
    histograms[i] = get_histogram(result)          
    

  return histograms.flatten()

In [78]:
def feature_extraction(image, N , R):
  # Obtenemos las dimensiones de la imagen
  height, width = image.shape[:2]  

  #print("IMAGEN NxM: ",height, "x",width )
 
  # Calculamos el número de bloques en filas y columnas
  num_blocks_rows = height // N
  num_blocks_cols = width // N

  # Dividimos la imagen en bloques de N filas
  blocks = np.vsplit(image, num_blocks_rows)

  # Creamos una lista para guardar todas las celdas
  celdas = []

  # Iteramos sobre los bloques
  for i, block in enumerate(blocks):
    # Dividimos cada bloque en bloques de N columnas
    cells = np.hsplit(block, num_blocks_cols)

    # Iteramos sobre las celdas
    for j, cell in enumerate(cells):
      # Añadimos cada celda a la lista
      celdas.append(cell)
  
  celdas = np.array(celdas)
  #print("TENEMOS: ",len(celdas), " celdas")
  histograms = np.zeros((len(celdas) , N*N))

  for i, celda in enumerate(celdas):    
    vecinos = im_to_col(celda,R,R).T
    # Inicializamos el array de resultados para todos los vecinos
    result = np.zeros(vecinos.shape[0], np.uint8)
    # Iteramos sobre cada vecino
    for j, v in enumerate(vecinos):
      v = v.reshape(R,R)  
      # Obtenemos el valor del píxel central
      center_pixel = v[1,1]
      # Inicializamos el valor binario a 0
      binary = ''   
      # Iteramos sobre los vecinos de la celda, excluyendo el píxel central
      for x in range(R):
        for y in range(R):  
          if x == 1 and y == 1:
            continue             
          if v[x, y] < center_pixel:
            binary += '0'
          else:
            binary += '1'
      # Modificamos el valor del píxel central por el del número binario obtenido
      result[j] = int(binary,2)

    # Calculamos el histograma del bloque
    histograms[i] = get_histogram(result)          
    

  return histograms.flatten()


In [62]:
# Cargamos la imagen
N = 16
R = 3
image = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)# Convertimos la imagen a escala de grises

result = feature_extraction2(image, N ,R)

print(len(result))


IMAGEN NxM:  128 x 128
TENEMOS:  64  celdas
16384


In [ ]:
datasets = ["cat_dog_100" , "cat_dog_500"]

HEIGHT = 384
WIDTH = 528

for dataset in datasets:
  print("dataset: ",dataset)
  # Iniciamos el contador de tiempo
  startTimer = time.perf_counter()

  # Definimos el directorio de entrenamiento y prueba
  train_dir = dataset+"/"+"train"
  test_dir = dataset+"/"+"test"

  # Inicializamos los arrays de datos de entrenamiento y prueba
  X_train = []
  y_train = []
  X_test = []
  y_test = []

  print("Procesamos las imágenes de entrenamiento")
  # Procesamos las imágenes de entrenamiento
  for label in os.listdir(train_dir):
    label_dir = os.path.join(train_dir, label)
    print("Procesando label: ",label)
    for count , image_filename in enumerate(os.listdir(label_dir)):
      image_path = os.path.join(label_dir, image_filename)
      image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
      resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
      features = feature_extraction(resized_image, N, R)
      X_train.append(features)
      y_train.append(label)

      progress = 100 * (count + 1) / len(os.listdir(label_dir))
      sys.stdout.write("\r Procesando Xtrain ... (" + str(int(progress)) + " %)")
    print("")

  

  print("Procesamos las imágenes de prueba")
  # Procesamos las imágenes de prueba
  for label in os.listdir(test_dir):
    label_dir = os.path.join(test_dir, label)
    print("Procesando label: ",label)
    for count , image_filename in enumerate(os.listdir(label_dir)):
      image_path = os.path.join(label_dir, image_filename)
      image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
      resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
      features = feature_extraction(resized_image, N, R)
      X_test.append(features)
      y_test.append(label)

      progress = 100 * (count + 1) / len(os.listdir(label_dir))
      sys.stdout.write("\r Procesando Xtest ... (" + str(int(progress)) + " %)")

  # Convertimos los datos de entrenamiento y prueba a arrays de NumPy
  X_train = np.array(X_train,np.uint8)
  y_train = np.array(y_train)
  X_test = np.array(X_test,np.uint8)
  y_test = np.array(y_test)


  # Guardamos los datos de entrenamiento y prueba en ficheros
  np.savetxt("datasets/"+dataset+"_X_train.txt", X_train,fmt="%u")
  np.savetxt("datasets/"+dataset+"_y_train.txt", y_train, fmt="%s")
  np.savetxt("datasets/"+dataset+"_X_test.txt", X_test,fmt="%u")
  np.savetxt("datasets/"+dataset+"_y_test.txt", y_test, fmt="%s")


  # Detenemos el contador de tiempo
  endTimer = time.perf_counter()

  # Calculamos el tiempo transcurrido
  elapsed = endTimer - startTimer
  # Obtenemos el tiempo transcurrido en minutos y segundos
  minutes, seconds = divmod(elapsed, 60)

  # Mostramos el tiempo transcurrido en minutos y segundos
  print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")
  

In [88]:
datasets = ["cat_dog_100" , "cat_dog_500"]

for dataset in datasets:
  print("dataset: ",dataset) 


  # Cargamos los datos de entrenamiento y prueba desde los ficheros
  X_train = np.loadtxt("datasets/"+dataset+"_X_train.txt" ,dtype=np.uint8)
  y_train = np.loadtxt("datasets/"+dataset+"_y_train.txt",dtype=str)
  X_test = np.loadtxt("datasets/"+dataset+"_X_test.txt" ,dtype=np.uint8)
  y_test = np.loadtxt("datasets/"+dataset+"_y_test.txt", dtype=str)
  
  

  # Creamos un clasificador SVM
  clf = SVC(gamma='auto')

  # Entrenamos el clasificador con los datos de entrenamiento
  clf.fit(X_train, y_train)


  # Realizamos predicciones con el clasificador entrenado
  y_pred = clf.predict(X_test)

  # Calculamos el porcentaje de acierto
  accuracy = np.mean(y_pred == y_test)
  print("Precisión: {:.2f}%".format(accuracy * 100))

dataset:  cat_dog_100


C:\Users\2012m\AppData\Local\Temp\ipykernel_11700\4182968165.py:8: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  X_train = np.loadtxt("datasets/"+dataset+"_X_train.txt" ,dtype=np.uint8)
C:\Users\2012m\AppData\Local\Temp\ipykernel_11700\4182968165.py:10: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.in

Precisión: 95.00%
dataset:  cat_dog_500


C:\Users\2012m\AppData\Local\Temp\ipykernel_11700\4182968165.py:8: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  X_train = np.loadtxt("datasets/"+dataset+"_X_train.txt" ,dtype=np.uint8)
C:\Users\2012m\AppData\Local\Temp\ipykernel_11700\4182968165.py:10: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.in

Precisión: 94.50%
